In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

### 네이버 영화

In [2]:
res = req.get('https://movie.naver.com/movie/point/af/list.naver')
res

<Response [200]>

In [3]:
res.text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\t\r\n\t\r\n\t\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\r\n\t\r\n\t\t\r\n\t\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\r\n\r\n\r\n\r\n\r\n\r\n<meta property="me2:image" content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag" content="네이버영화 "/>\r\n<meta property="me2:category1" content="네이버영화"/>\r\n<meta property="me2:category2" content=""/>\r\n\r\n\r\n\r\n\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t<meta property="og:title" content="평점 : 네이버 영화"/>\r\n\t\t<meta property="og:description" content="네티즌 평점과 리뷰 정보 제

In [4]:
soup = bs(res.text, "html.parser")

### 평점

In [11]:
score = soup.select('div.list_netizen_score > em')

In [16]:
len(score)

10

In [13]:
score_list = []
for i in score:
    score_list.append(i.text)

In [14]:
score_list

['8', '9', '1', '1', '10', '10', '10', '10', '9', '8']

### 영화 제목

In [59]:
title = soup.select('td.title > a.movie.color_b') # 띄어쓰기는 점으로 표시

In [61]:
for i in title:
    print(i.text)

우리에게 내일은 없다
공조2: 인터내셔날
비상선언
서울대작전
공조2: 인터내셔날
공조2: 인터내셔날
탑건: 매버릭
탑건: 매버릭
사랑할 땐 누구나 최악이 된다
탑건: 매버릭


In [62]:
title_list = []
for i in title:
    title_list.append(i.text)

In [63]:
len(title)
# title

10

### 리뷰

In [84]:
view = soup.select('td.title')
view_list = []
for i in view:
    review = i.text
    review = review.split('\n\n')[2]
    review = review.replace('\n', '')
    review = review.replace('\t', '')
    review = review.replace('신고', '')
    view_list.append(review)

In [86]:
len(view_list)

10

## 10페이지

In [109]:
from tqdm import tqdm_notebook as tqdm

In [110]:
page = range(1, 11) # 1페이지부터 10 페이지까지

In [111]:
total_title_list = []
total_review_list = []
total_score_list = []

for i in tqdm(page):
    url = 'https://movie.naver.com/movie/point/af/list.naver?&page={}'.format(i)
    print(url)
    
    res = req.get(url)
    soup = bs(res.text, 'html.parser')
    
    # 평점
    score = soup.select('div.list_netizen_score > em')
    for i in score:
        total_score_list.append(i.text)

    # 제목
    title = soup.select('td.title > a.movie.color_b')
    for i in title:
        total_title_list.append(i.text)

    # 리뷰
    view = soup.select('td.title')
    for i in view:
        review = i.text
        review = review.split('\n\n')[2]
        review = review.replace('\n', '')
        review = review.replace('\t', '')
        review = review.replace('신고', '')
        total_review_list.append(review)

C:\Users\82104\AppData\Local\Temp\ipykernel_12632\2251660559.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(page):


  0%|          | 0/10 [00:00<?, ?it/s]

https://movie.naver.com/movie/point/af/list.naver?&page=1
https://movie.naver.com/movie/point/af/list.naver?&page=2
https://movie.naver.com/movie/point/af/list.naver?&page=3
https://movie.naver.com/movie/point/af/list.naver?&page=4
https://movie.naver.com/movie/point/af/list.naver?&page=5
https://movie.naver.com/movie/point/af/list.naver?&page=6
https://movie.naver.com/movie/point/af/list.naver?&page=7
https://movie.naver.com/movie/point/af/list.naver?&page=8
https://movie.naver.com/movie/point/af/list.naver?&page=9
https://movie.naver.com/movie/point/af/list.naver?&page=10


In [112]:
len(total_title_list), len(total_review_list), len(total_score_list)

(100, 100, 100)

### 데이터 프레임화 하기

In [113]:
data = {'영화 제목' : total_title_list, '평점' : total_score_list, '리뷰' : total_review_list}

In [114]:
movie = pd.DataFrame(data)
movie

,영화 제목,평점,리뷰
0,토르: 러브 앤 썬더,1,막판에 초글링 싸움에 토린이 데뷔는 뭐냐
1,헌트,10,이정재배우의 감독변신재밌는영화
2,육사오(6/45),10,연기너무잘하시고너무웃김 재밌음 맘편안히 웃고오세요
3,리미트,3,마지막에 산에서 싸우려고 스토리가 산으로 가는거였구나..
4,공조2: 인터내셔날,10,액션도 너무 좋고 재미있었습니다
...,...,...,...
95,육사오(6/45),10,재밌어요
96,불도저에 탄 소녀,10,김혜윤 연기력 무엇!!
97,정직한 후보,10,재밌어서 이걸 왜 이제 봤나 싶었음. 2편 기대 중
98,육사오(6/45),7,가볍게 보기 좋은 오락영화!
